# Prédiction de la consommation d'électricité en France :

Projet developpement logiciel 

Traîtement des données sur Jupyter Notebook (distribution Anaconda) 

Etude réalisée en language Python 

Source des données : [Données annuelles de la consommation brute d'électricité du 1er Janvier 2012 au 31 mai 2022](https://www.data.gouv.fr/fr/datasets/pic-journalier-de-la-consommation-brute-delectricite-janvier-2012-a-septembre-2022/)


In [2]:
# Importer les librairies nécessaires:

import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from ipywidgets import widgets, interact, interactive, fixed, interact_manual
import statsmodels.api as sm
import pooch
from IPython import get_ipython
import seaborn as sns
import time
from datetime import datetime
import itertools

import pmdarima  ## utilisé pour l'application de la fonction auto_arima
from pmdarima import auto_arima

from statsmodels.tsa.seasonal import (
    seasonal_decompose,
)  # décomposer la série temporelle
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import (
    adfuller,
)  # tester  la stationnarité de la série temporelle
from pandas.plotting import register_matplotlib_converters
from sklearn.metrics import mean_squared_error
import pylab
from pylab import rcParams  # paramètres de style
import statsmodels.api as sm
import warnings  # pour se débarasser des warnings

warnings.filterwarnings("ignore")

from prophet import Prophet

register_matplotlib_converters()
color_pal = sns.color_palette()

%matplotlib inline

In [3]:
# Versions utilisées :
print("Pandas : " + str(pd.__version__))
print("Numpy : " + str(np.__version__))
print("Matplotlib : " + str(matplotlib.__version__))
print("Seaborn : " + str(sns.__version__))
print("Statsmodels : " + str(sm.__version__))

Pandas : 1.4.4
Numpy : 1.23.3
Matplotlib : 3.5.2
Seaborn : 0.11.2
Statsmodels : 0.13.5


In [4]:
# Paramètre de style :
pylab.style.use("fivethirtyeight")
params = {
    "legend.fontsize": "x-large",
    "figure.figsize": (20, 6),
    "lines.linewidth": 1.8,
    "axes.labelsize": "x-large",
    "axes.titleweight": "bold",
    "xtick.labelsize": "x-large",
    "ytick.labelsize": "x-large",
}
pylab.rcParams.update(params)

## 1- Création de la base de données :

### 1.1 - Téléchargement des données :

In [ ]:
# Création du fichier 'consommation.csv' à partit de l'url

url = "https://www.data.gouv.fr/fr/datasets/r/72c72414-a2d8-4dc5-b699-ff70eb6b4c4c"
path_target = "./consommation.csv"
path, fname = os.path.split(path_target)
pooch.retrieve(url, path=path, fname=fname, known_hash=None)

# Chargement du dataset "consommation.csv"
data = pd.read_csv(
    "consommation.csv",
    delimiter=";",
    comment="#",
    na_values="n/d",
    parse_dates=["date"],
    converters={"heure": str},
)
data

### 1.2 - Nettoyage des données

In [ ]:
# Restriction des données sur les modalités "date", heure" et "consommation"

df = data.copy()
df = data[["date", "heure", "consommation"]]
df = df.rename(
    columns={"date": "Date", "heure": "Heure", "consommation": "Consommation"}
)
df = df.dropna(axis=0)  # supprimer les valeurs manquantes
df = df.set_index("Date")
df.index = pd.to_datetime(df.index)  # convertir l'objet 'Datetime' de string à datetime
df = df.sort_values(
    by=["Date", "Heure"], ascending=(True, True)
)  # trier le dataframe dans l'ordre croissant

In [ ]:
# description des données
df.describe()

### 1.3 Aperçu des données :

In [ ]:
# Visualisation des données :
df.plot()
plt.xlabel("Date")
plt.ylabel("Consommation")
plt.title("Consommation d'électricité en France (MW)")
plt.show()
# plt.savefig("Consommation_elec.pdf")

In [ ]:
df.head(10)

In [ ]:
df.tail(10)

# 2- Principe du modèle ARIMA:

## Qu'est ce qu'une série temporelle? 

une série temporelle est une suite finie $(x_1, ..., x_n)$ de données indexée par le temps, l'indice de temps peut être le jour, l'année etc, le nombre $n$ est appelé la longueur de la série.

Une série temporelle peur être décomposée en 3 composantes:

1. Tendance : 
elle montre une direction générale des données de la série chronologique sur une longue période. Une tendance peut être croissante , décroissante ou horizontale (stationnaire).

2. Saisonnalité : 
la composante saisonnière présente une tendance qui se répète en ce qui concerne le moment, la direction et l’ampleur, le chocolat par exemple est un produit saisonnier connaîssant de fortes ventes durant Pâcques , Noël et une réduction de ventes durant l'été.

3. Bruit :
qui représente des pics et des creux à intervalles aléatoires.

### Qu'est ce qu'une série stationnaire?

Une série est stationnaire si :

1. La moyenne et la variance de la série ne varient pas dans le temps .

2. La covariance du i-ème terme et du (i+m) -ième terme n'est fonction du temps.



### Pourquoi avoir choisi le modèle ARIMA??

ARIMA est un modèle statistique conçu pour l'analyse et la prédiction des données d'une série temporelle , il permet de déterminer les valeurs intégrées à cette dernière en fonction des précédentes observations , c'est pour cette raison que nous avons choisi ce modèle pour notre projet.


### Paramètres du modèle :

Le modèle ARIMA possède 3 paramètres :p, d et q , il s'écrit de la manière suivante : ARIMA(p, d , q) où:

p : le nombre de décalage qu'il faudra considérer pour le modèle autoregressif .

d : le nombre de fois qu'il faut différentier la série afin de la rendre stationnaire (vous verrez ci-dessus que d=0 car notre processus est déjà stationnaire).

q : l'ordre du modèle MA.


## 2.1 - Aggrégation des données :

In [ ]:
len(df)

Nous allons Aggréger les données en prenant la moyenne de la consommation d'électricité par jour.

In [ ]:
try:
    weekly_cons = df1.groupby(["Consommation", pd.Grouper(key="Date", freq="D")]).size()
    weekly_cons = weekly_cons.unstack(0)
except:
    print("\n Here seems to be one issue")

In [ ]:
df1 = df[["Consommation"]].resample("D").mean()
df1 = pd.DataFrame(df1)  # convertir l'objet ts en dataframe
df1.tail(10)

In [ ]:
len(df1)

In [ ]:
plt.plot(df1)
plt.xlabel("Date")
plt.ylabel("Consommation (MW)")
plt.title("Moyenne de la consommation d'électricité par jour")
plt.show()
# plt.savefig("conso_hebdo.pdf")

In [ ]:
decomp = sm.tsa.seasonal_decompose(df1, model="additive")
fig = decomp.plot()
plt.show()
# plt.savefig("decomp.pdf")

## Test de vérification de la stationnalité de la série temporelle

Afin de tester la stationnalité de la série , nous avons deux méthodes :

1. À l'oeil nu en traçant la moyenne et l'écart type mobile . 

2. En appliquant le test de Dickey-Fuller (nous verrons que c'est une méthode qui prends du temps..)

Si la valeur du paramètre p est inférieure ou égale à 0.05 alors d=0 et notre série est stationnaire (nous rejetons l'hypothèse nulle sinon elle ne l'est pas . 


À partir du package statsmodels, la fonction de test dickey-fuller augmentée est importée. Il renvoie un tuple composé des valeurs : adf, pvalue, usedlag, nobs, critical values etc

Nous allons commencer par définir une fonction qui nous permet d'effectuer tous ces tests :

In [ ]:
def get_stationarity(timeseries):
    # Statistiques mobiles
    rolling_mean = timeseries.rolling(window=12).mean()
    rolling_std = timeseries.rolling(window=12).std()

    # tracé statistiques mobiles
    original = plt.plot(timeseries, color="blue", label="Données originales")
    mean = plt.plot(rolling_mean, color="red", label="Moyenne Mobile")
    std = plt.plot(rolling_std, color="green", label="Ecart-type Mobile")
    plt.legend(loc="best")
    plt.title("Moyenne et écart-type Mobiles")
    plt.show(block=False)

    # Test Dickey–Fuller :
    res = adfuller(timeseries["Consommation"])
    print("Statistiques ADF : {}".format(res[0]))
    print("p-value : {}".format(res[1]))
    print("Valeurs Critiques :")
    for key, value in res[4].items():
        print("\t{}: {}".format(key, value))

In [ ]:
# Déf d'une fonction qui teste la valeur de p:
def test_val_p(data):
    fuller_test = adfuller(data)
    print("La valeur de p est:", fuller_test[1])
    if fuller_test[1] <= 0.05:
        print("la série est stationaire")
    else:
        print("la série n'est pas stationaire")

In [ ]:
get_stationarity(df1)
plt.savefig("moy_ecart_type.pdf")

In [ ]:
test_val_p(df1)

 ## Construction du modèle ARIMA:

### Comment obtenir les valeurs de p, d ,q : nous allons utiliser la fonction auto_arima

In [ ]:
pip install pmdarima 

In [ ]:
# Nous voulons connaître le nombre de combinaisons possibles entre p, d et q:
p = range(0, 8)
d = range(0, 8)
q = range(0, 2)

pdq_comb = list(itertools.product(p, d, q))
len(pdq_comb)

In [ ]:
# Création et test du modèle:
train = df1.loc[df1.index < "2020-12-08"]
test = df1.loc[df1.index >= "2020-12-08"]

In [ ]:
len(df1)

In [ ]:
# Visualisation train/test split :
fig, ax = plt.subplots(figsize=(15, 5))
train.plot(ax=ax, label="Training Set", title="Data Train/Test Split")
test.plot(ax=ax, label="Test Set")
ax.axvline("2020-12-08", color="green", ls="--")
ax.legend(["Modèle", "Test"])
plt.show()
# plt.savefig("data_train_tes.pdf")

In [ ]:
import pmdarima

from pmdarima import auto_arima

auto_arima(
    train,
    m=12,
    start_P=0,
    seasonal=True,
    trace=True,
    error_action="ignore",
    suppress_warnings=True,
    stepwise=True,
)

 ARIMA(2,1,1)(2,0,1)[12] intercept   : AIC=62147.813, Time=22.90 sec
 ARIMA(2,1,1)(2,0,0)[12] intercept   : AIC=62196.598, Time=16.89 sec
 ARIMA(2,1,1)(2,0,2)[12] intercept   : AIC=inf, Time=34.63 sec
 ARIMA(2,1,1)(1,0,2)[12] intercept   : AIC=inf, Time=19.45 sec
 ARIMA(1,1,1)(2,0,1)[12] intercept   : AIC=62485.417, Time=7.41 sec
 ARIMA(2,1,0)(2,0,1)[12] intercept   : AIC=62373.094, Time=8.01 sec
 ARIMA(3,1,1)(2,0,1)[12] intercept   : AIC=62115.382, Time=13.73 sec
 ARIMA(3,1,1)(1,0,1)[12] intercept   : AIC=62113.418, Time=3.01 sec
 ARIMA(3,1,1)(0,0,1)[12] intercept   : AIC=62127.121, Time=2.30 sec
 ARIMA(3,1,1)(1,0,0)[12] intercept   : AIC=62111.590, Time=2.40 sec
 ARIMA(3,1,1)(0,0,0)[12] intercept   : AIC=62448.299, Time=1.29 sec
 ARIMA(3,1,1)(2,0,0)[12] intercept   : AIC=62113.438, Time=7.48 sec
 ARIMA(3,1,0)(1,0,0)[12] intercept   : AIC=62415.145, Time=1.62 sec
 ARIMA(4,1,1)(1,0,0)[12] intercept   : AIC=61947.918, Time=3.17 sec
 ARIMA(4,1,1)(0,0,0)[12] intercept   : AIC=62131.326, T

# 3- Application du modèle ARIMA au données:

In [ ]:
decomposition = seasonal_decompose(df1)
model = ARIMA(df1, order=(5, 1, 3), seasonal_order=(0, 0, 2, 12))
results = model.fit()
plt.plot(df1)
plt.plot(results.fittedvalues, color="red")
plt.title("Prédiction sur les données")
plt.show()

## Application du modèle au test:

In [ ]:
decomposition = seasonal_decompose(df1)
model = ARIMA(test, order=(5, 1, 3), seasonal_order=(0, 0, 2, 12))
results = model.fit()
plt.plot(df1)
plt.plot(results.fittedvalues, color="red")
plt.title("Prédiction sur le test")
plt.legend(["Vrai données", "Prédictions"])

In [ ]:
df1.tail(10)

In [ ]:
# prédiction du 2022-06-01 jusqu'au 2022-12-08 : 192 jours
from pandas.tseries.offsets import DateOffset

new_dates = [df1.index[-1] + DateOffset(days=x) for x in range(1, 192)]
df1_pred = pd.DataFrame(index=new_dates, columns=df1.columns)
df1_pred

In [ ]:
len(df1)

Le modèle ARIMA prend comme arguments le début et la fin de l'index énuméré et non la plage de dates.

Nous avons créé un datframe vide ayant des index de dates futures et nous allons les concaténer avec notre dataframe d'origine.

Notre dataframe avait 3804 lignes et le nouveau possède 199 lignes, on a au total 4004 lignes.

Par conséquent, pour obtenir les prédictions uniquement pour les données futures, nous allons prédire de la ligne 3805 à 4004 .

In [ ]:
df2 = pd.concat([df1, df1_pred])

df2["predictions"] = results.predict(start=3805, end=4004)
df2[["Consommation", "predictions"]].plot()
df2.tail()

In [ ]:
len(df1_pred)

In [ ]:
results.resid

In [ ]:
results.resid.plot()
plt.title("Résidus de la prédiction")

In [ ]:
results.resid.plot(kind="kde")

Cette méthode n'a pas vraiment abouti à grand chose , après plusieurs recherches , nous avons trouvé un outil très puissant nommé "prophet" à partir duquel nous allons nous appuyer dans la suite de notre raisonnement.

## Autre méthode : utiliser le package prophet

Prophet est un package qui permet d'effectuer des prévisions des données de séries temporelles basées sur un modèle additif.

Nous allons à présent travailler avec [les données de la consommation d'électricité en France du 01 Juin 2022 jusqu'au 29 novembre 2022](https://odre.opendatasoft.com/explore/dataset/eco2mix-national-tr/download/?format=csv&disjunctive.nature=true&q=date_heure:%5B2022-05-31T22:00:00Z+TO+2022-11-29T22:59:59Z%5D&timezone=Europe/Berlin&lang=fr&use_labels_for_header=true&csv_separator=%3B) 

In [ ]:
# Téléchargement des données de 2022:
url2 = "https://odre.opendatasoft.com/explore/dataset/eco2mix-national-tr/download/?format=csv&disjunctive.nature=true&q=date_heure:%5B2022-05-31T22:00:00Z+TO+2022-11-29T22:59:59Z%5D&timezone=Europe/Berlin&lang=fr&use_labels_for_header=true&csv_separator=%3B"
path_target = "./consommation_2022.csv"
path, fname = os.path.split(path_target)
pooch.retrieve(url2, path=path, fname=fname, known_hash=None)

# Chargement du dataset "consommation.csv"
data1 = pd.read_csv(
    "consommation_2022.csv",
    delimiter=";",
    comment="#",
    na_values="n/d",
    parse_dates=["Date"],
    converters={"heure": str},
)
data1

Afin d'appliquer prophet , notre adataframe doit avoir une forme spécifique :

la première colonne doit porter le nom 'ds' et contenir les dates et la deuxième colonne doit porter le nom de 'y' et contenir ce que l'on veut prédire , dans notre cas 'consommation'.

In [ ]:
# Restriction des données sur les modalités "date " ," heure" et "consommation"
df2 = data1.copy()
df2 = data1[["Date", "Heure", "Consommation (MW)"]]
df2 = df2.rename(columns={"Date": "ds", "Consommation (MW)": "y"})
df2 = df2.dropna()  # supprimer les valeurs aberrantes
df2 = df2.sort_values(
    by=["ds", "Heure"], ascending=(True, True)
)  # ordonner les colonnes 'ds' et 'Heure' dans l'ordre croissant
df2["ds"] = pd.to_datetime(df2["ds"])  # convertir l'objet 'ds' en datetime

In [ ]:
df2.head(10)

In [ ]:
df2.tail(10)

In [ ]:
# Test du modèle :
model = Prophet()
model.fit(df2)
future = model.make_future_dataframe(periods=10, freq="15min", include_history=False)
forecast = model.predict(future)

In [ ]:
forecast

In [ ]:
forecast[["ds", "yhat"]]

In [ ]:
# Prédiction sur 10 jours à partir du 29 novembre :
m = Prophet()
m.fit(df2)  # ajuster notre modèle 'm' sur l'ensemble des données
f = model.make_future_dataframe(periods=96 * 10, freq="15min", include_history=False)
predic = model.predict(f)

In [ ]:
s = predic[["ds", "yhat"]]  # la colonne 'yhat' contient les prédiction
s

In [ ]:
predic_finale = s[len(s) - 97 : 959]
predic_finale

In [ ]:
predic_finale = predic_finale.rename(
    columns={"ds": "Date et Heure", "yhat": "Consommation (MW)"}
)

In [ ]:
predic_finale

In [ ]:
dataframe = pd.DataFrame(predic_finale)

### Création du dataframe 'prediction.csv'

In [ ]:
dataframe.to_csv(path_or_buf="./prediction.csv", sep=";")